In [1]:
#library import
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
import os, random, json
spark=SparkSession.builder.master("local").appName("studentmarkpractice").getOrCreate()

In [2]:
#script execution for raw data to pre-landing
class Raw_to_prelanding:
    
    """Script to read raw file from local path to pre landing.."""
    
    def __init__(self,file_parameter):
        
        """function to get input parameters to read the file from local path"""
        self.file_parameter=file_parameter

    def read_csv(self):
        return spark.read.format(self.file_parameter['src_format']).option("header","True").load(self.file_parameter['src_path'])
    
    def write_csv(self,df):
        df.write.format(self.file_parameter['dest_format']).mode("overwrite").save(self.file_parameter['dest_path'])
        
    def create_view(self,df):
        df.createOrReplaceTempView(self.file_parameter['view_name'])
        
    def open_dir(self):
        os.chdir(self.file_parameter['dest_path'])
        dest_file_path=f"{self.file_parameter['view_name']}.csv"
        for file in os.listdir():
        # Check whether file is in text format or not
            if file.endswith(".csv"):
                src_file_path = f"{file}"
                os.rename(src_file_path,dest_file_path)
            else:
                src_file_path = f"{file}"
                os.remove(src_file_path)
        
    def dynamic_data_generate(self):
        studentid=spark.sql("select Student_ID from studentdetails")
        studentcount=studentid.count()
        subjectid=spark.sql("select Subject_ID from subjectdetails")
        subjectcount=subjectid.count()
        month = ["11-26-2021","12-26-2021","01-26-2022"]
        datastr="["

        for x in range(studentcount):

            for y in month:

                for z in range(subjectcount-1):
                    datastr=datastr+'{"Student_ID":'+studentid.collect()[x][0]+',"Date":"'+y+'","Subject_ID":'+subjectid.collect()[z][0]+',"Marks":'+str(random.randint(40,100))+'},'
        datastr=datastr+'{"Student_ID":'+studentid.collect()[x][0]+',"Date":"'+y+'","Subject_ID":'+subjectid.collect()[z][0]+',"Marks":'+str(random.randint(40,100))+'}'        
        datastr=datastr+"]"
        
        data=json.loads(datastr)
        df = spark.createDataFrame(data)
        self.write_csv(df)
        self.open_dir()
    
    def main_block(self):
        
        """main execution block"""
        df=self.read_csv() 
        self.write_csv(df)
        self.create_view(df)
        self.open_dir()
        


In [4]:
if __name__=='__main__':
    file_parameter1={
        "dest_format":"csv",
        "src_path":"C:/Git_Repo/big_data_project/raw_data/raw_files/school/studentdetails.csv",
        "src_format":"csv",
        "dest_path":"C:/Git_Repo/big_data_project/data_files/pre_landing/raw_files/school/student",
        "view_name":"studentdetails"
    }
    
    file_parameter2={
        "dest_format":"csv",
        "src_path":"C:/Git_Repo/big_data_project/raw_data/raw_files/school/subjectdetails.csv",
        "src_format":"csv",
        "dest_path":"C:/Git_Repo/big_data_project/data_files/pre_landing/raw_files/school/subject",
        "view_name":"subjectdetails"
    }
    
    file_parameter3={
        "dest_format":"csv",
        "dest_path":"C:/Git_Repo/big_data_project/data_files/pre_landing/raw_files/school/marks",
        "view_name":"markdetails"
    }
    
    prelanding_obj1=Raw_to_prelanding(file_parameter1)
    prelanding_obj2=Raw_to_prelanding(file_parameter2)
    prelanding_obj3=Raw_to_prelanding(file_parameter3)
    prelanding_obj1.main_block()
    prelanding_obj2.main_block()
    prelanding_obj3.dynamic_data_generate()